In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle 
from xgboost import XGBClassifier
from time import time


In [2]:
def evaluation_map(actual, predicted):
#     if len(predicted)>k:
#         predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if num_hits == 0.0:
        return 0.0

    return score / num_hits

In [3]:
to_test_1 = pd.read_pickle("to_test_1.p")
to_test_2 = pd.read_pickle("to_test_2.p")

In [9]:
# model31_v = pd.read_pickle("model3_1_tuning.sav")
# model31_t = pd.read_pickle("model3_1_tuning_final.sav")


model1 = pd.read_pickle("model35_tuning_.sav")
model2 = pd.read_pickle("model35_tuning_final.sav")
model3 = pd.read_pickle("model36_tuning_.sav")
model4 = pd.read_pickle("model36_tuning_final.sav")
# model5 = pd.read_pickle("model34_tuning_.sav")
# model6 = pd.read_pickle("model34_tuning_final.sav")




Validation Round

In [5]:
def get_map(model,stage):
    
    if stage == "validation":
        x_test,y_test = to_test_1.iloc[:,2:-1], to_test_1.iloc[:,-1]
        y_pred_1 = model.predict_proba(x_test)

        pred_no = []
        pred_yes =[]
        for prediction in y_pred_1:
            pred_no.append(prediction[0])
            pred_yes.append(prediction[1])

        x_test['pred_no'] = pred_no
        x_test['pred_yes'] = pred_yes

        x_test['user_id'] = to_test_1['user_id']
        x_test['product_id'] = to_test_1['product_id']
        
        
        total = 0
        user_tested = 0
        
        for user in range(1,501):

            true_reordered = to_test_1.query('user_id=={} and reordered==1'.format(user)).product_id.to_list()
            # if the user has no reorder in the next order, do not consider. 
            if len(true_reordered) == 0:
                continue

            user_tested +=1 

            predict_top_5 = x_test[x_test['user_id'] ==user].nlargest(5, 'pred_yes')['product_id'].to_list()

            total += evaluation_map(true_reordered, predict_top_5)
            
        return (total/user_tested)
    
    
    else:
        x_test,y_test = to_test_2.iloc[:,2:-1], to_test_2.iloc[:,-1]
        y_pred_1 = model.predict_proba(x_test)

        pred_no = []
        pred_yes =[]
        for prediction in y_pred_1:
            pred_no.append(prediction[0])
            pred_yes.append(prediction[1])

        x_test['pred_no'] = pred_no
        x_test['pred_yes'] = pred_yes

        x_test['user_id'] = to_test_2['user_id']
        x_test['product_id'] = to_test_2['product_id']
        
        
        total = 0
        user_tested = 0
        
        for user in range(1,501):

            true_reordered = to_test_2.query('user_id=={} and reordered==1'.format(user)).product_id.to_list()
            # if the user has no reorder in the next order, do not consider. 
            if len(true_reordered) == 0:
                continue

            user_tested +=1 

            predict_top_5 = x_test[x_test['user_id'] ==user].nlargest(5, 'pred_yes')['product_id'].to_list()

            total += evaluation_map(true_reordered, predict_top_5)
            
        return (total/user_tested)

In [10]:
print("========= model35 validdation and test ==========")
print(get_map(model1,"validation"))
print(get_map(model2,"test"))

print("========= model36 validdation and test ==========")
print(get_map(model3,"validation"))
print(get_map(model4,"test"))

# print("========= model34 validdation and test ==========")
# print(get_map(model5,"validation"))
# print(get_map(model6,"test"))



========= model35 validdation and test ==========
0.5763373362445411
0.5101789709172259
========= model36 validdation and test ==========
0.5763373362445411
0.5047119686800892
